# 09 indep_winsize_5 Clustering and Insight Workflow

Unsupervised clustering workflow focused on `label_indep_winsize_5.npy`.

Approach:
- Feature-based clustering (`KMeans` + `Agglomerative Ward`) with model selection on `k=2..6`.
- DTW-based clustering on resampled trajectories for shape validation.
- Cross-method agreement + representative books + narrative insights.

This run uses `MA_WINDOW=5` for smoothing-derived features and centroid trajectory plots.


In [ ]:
from __future__ import annotations

from pathlib import Path
from itertools import combinations
import json

import numpy as np
import pandas as pd

import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.decomposition import PCA
from sklearn.metrics import (
    silhouette_score,
    davies_bouldin_score,
    calinski_harabasz_score,
    adjusted_rand_score,
    normalized_mutual_info_score,
)
from scipy.cluster.hierarchy import linkage, dendrogram
from scipy.spatial.distance import squareform

SEED = 42
rng = np.random.default_rng(SEED)
np.random.seed(SEED)

PROJECT_ROOT = Path('.').resolve()
DATA_DIR = PROJECT_ROOT / 'data'
PROCESSED_DIR = DATA_DIR / 'processed'
METADATA_PATH = DATA_DIR / 'metadata.csv'

OUT_ROOT = PROJECT_ROOT / 'outputs' / 'excitement_indep_clustering'
FIG_DIR = OUT_ROOT / 'figures'
TABLE_DIR = OUT_ROOT / 'tables'
for d in [OUT_ROOT, FIG_DIR, TABLE_DIR]:
    d.mkdir(parents=True, exist_ok=True)

LABEL_FILE = 'label_indep_winsize_5.npy'
EMB_FILE = 'embeddings.npy'

K_RANGE = list(range(2, 7))
N_STABILITY_SEEDS = 20
L_DTW = 200
MA_WINDOW = 5
EPS = 1e-8

print(f'PROJECT_ROOT={PROJECT_ROOT}')
print(f'OUT_ROOT={OUT_ROOT}')
print(f'LABEL_FILE={LABEL_FILE}, K_RANGE={K_RANGE}, L_DTW={L_DTW}, MA_WINDOW={MA_WINDOW}')


In [ ]:
def normalize_label_shape(y: np.ndarray, label_path: Path) -> np.ndarray:
    y = np.asarray(y)
    if y.ndim == 1:
        return y
    if y.ndim == 2 and 1 in y.shape:
        return y.reshape(-1)
    raise ValueError(f'Unsupported label shape {y.shape} at {label_path}')


def moving_average_1d(x: np.ndarray, window: int) -> np.ndarray:
    x = np.asarray(x, dtype=np.float64).reshape(-1)
    if window <= 1:
        return x.copy()
    if window % 2 == 0:
        raise ValueError(f'MA window must be odd, got {window}')
    return pd.Series(x).rolling(window=window, center=True, min_periods=1).mean().to_numpy()


def safe_corr(a: np.ndarray, b: np.ndarray) -> float:
    a = np.asarray(a, dtype=np.float64).reshape(-1)
    b = np.asarray(b, dtype=np.float64).reshape(-1)
    if len(a) < 2 or len(b) < 2:
        return 0.0
    if np.std(a) < EPS or np.std(b) < EPS:
        return 0.0
    c = np.corrcoef(a, b)[0, 1]
    if not np.isfinite(c):
        return 0.0
    return float(c)


def dtw_distance(a: np.ndarray, b: np.ndarray) -> float:
    a = np.asarray(a, dtype=np.float64).reshape(-1)
    b = np.asarray(b, dtype=np.float64).reshape(-1)
    n, m = len(a), len(b)
    dp = np.full((n + 1, m + 1), np.inf, dtype=np.float64)
    dp[0, 0] = 0.0
    for i in range(1, n + 1):
        ai = a[i - 1]
        for j in range(1, m + 1):
            cost = abs(ai - b[j - 1])
            dp[i, j] = cost + min(dp[i - 1, j], dp[i, j - 1], dp[i - 1, j - 1])
    return float(dp[n, m])


def resample_to_len(y: np.ndarray, L: int) -> np.ndarray:
    y = np.asarray(y, dtype=np.float64).reshape(-1)
    if len(y) == L:
        return y.copy()
    x_old = np.linspace(0.0, 1.0, len(y))
    x_new = np.linspace(0.0, 1.0, L)
    return np.interp(x_new, x_old, y)


def compute_features(y: np.ndarray, ma_window: int) -> dict:
    y = np.asarray(y, dtype=np.float64).reshape(-1)
    T = len(y)
    d = np.diff(y)
    abs_d = np.abs(d) if T > 1 else np.array([], dtype=np.float64)

    p10 = float(np.quantile(y, 0.10))
    p90 = float(np.quantile(y, 0.90))
    q25 = float(np.quantile(y, 0.25))
    q75 = float(np.quantile(y, 0.75))

    counts = np.bincount(y.astype(int), minlength=5)
    probs = counts / counts.sum()
    entropy = float(-np.sum(probs[probs > 0] * np.log2(probs[probs > 0])))

    if T > 1:
        up_rate = float(np.mean(d > 0))
        down_rate = float(np.mean(d < 0))
        flat_rate = float(np.mean(d == 0))
        jump_ge_2_rate = float(np.mean(abs_d >= 2))
        lag1_autocorr = safe_corr(y[:-1], y[1:])

        nz = d != 0
        if np.sum(nz) >= 2:
            signs = np.sign(d[nz])
            sign_change_rate = float(np.mean(signs[1:] != signs[:-1]))
        else:
            sign_change_rate = 0.0
    else:
        up_rate = down_rate = flat_rate = 0.0
        jump_ge_2_rate = 0.0
        lag1_autocorr = 0.0
        sign_change_rate = 0.0

    t_norm = np.linspace(0.0, 1.0, T) if T > 1 else np.zeros(T, dtype=np.float64)
    corr_with_position = safe_corr(y, t_norm)
    if T > 1:
        slope_position = float(np.polyfit(t_norm, y, deg=1)[0])
    else:
        slope_position = 0.0

    b0, b1, b2, b3 = np.linspace(0, T, 4, dtype=int)
    seg_early = y[b0:b1] if b1 > b0 else y
    seg_mid = y[b1:b2] if b2 > b1 else y
    seg_late = y[b2:b3] if b3 > b2 else y

    y_ma = moving_average_1d(y, ma_window)

    out = {
        'T': int(T),
        'mean_y': float(np.mean(y)),
        'std_y': float(np.std(y)),
        'median_y': float(np.median(y)),
        'iqr_y': float(q75 - q25),
        'min_y': float(np.min(y)),
        'max_y': float(np.max(y)),
        'p10_y': p10,
        'p90_y': p90,
        'range_y': float(np.max(y) - np.min(y)),
        'prop_label_0': float(probs[0]),
        'prop_label_1': float(probs[1]),
        'prop_label_2': float(probs[2]),
        'prop_label_3': float(probs[3]),
        'prop_label_4': float(probs[4]),
        'entropy_labels': entropy,
        'mean_abs_diff': float(np.mean(abs_d)) if len(abs_d) else 0.0,
        'std_diff': float(np.std(d)) if len(d) else 0.0,
        'p95_abs_diff': float(np.quantile(abs_d, 0.95)) if len(abs_d) else 0.0,
        'jump_ge_2_rate': jump_ge_2_rate,
        'up_rate': up_rate,
        'down_rate': down_rate,
        'flat_rate': flat_rate,
        'lag1_autocorr': lag1_autocorr,
        'sign_change_rate': sign_change_rate,
        'corr_with_position': corr_with_position,
        'slope_position': slope_position,
        'mean_early': float(np.mean(seg_early)),
        'mean_mid': float(np.mean(seg_mid)),
        'mean_late': float(np.mean(seg_late)),
        'mean_ma5': float(np.mean(y_ma)),
        'std_ma5': float(np.std(y_ma)),
        'p95_ma5': float(np.quantile(y_ma, 0.95)),
    }
    return out


def expected_outputs() -> list[Path]:
    files = [
        TABLE_DIR / 'indep_book_features.csv',
        TABLE_DIR / 'indep_book_features_zscore.csv',
        TABLE_DIR / 'cluster_quality_by_method.csv',
        TABLE_DIR / 'cluster_assignments_feature.csv',
        TABLE_DIR / 'cluster_assignments_dtw.csv',
        TABLE_DIR / 'cluster_profile_summary.csv',
        TABLE_DIR / 'cluster_representatives.csv',
        TABLE_DIR / 'cluster_method_agreement.csv',
        TABLE_DIR / 'kmeans_elbow_curve.csv',
        TABLE_DIR / 'genre_by_feature_cluster_counts.csv',
        TABLE_DIR / 'genre_by_feature_cluster_proportions.csv',
        TABLE_DIR / 'feature_cluster_signature_top_features.csv',
        TABLE_DIR / 'figure_legend_checks.csv',
        TABLE_DIR / 'integrity_checks.csv',
        FIG_DIR / 'feature_correlation_heatmap.png',
        FIG_DIR / 'feature_pca_scatter_feature_clusters.png',
        FIG_DIR / 'feature_elbow_kmeans_inertia.png',
        FIG_DIR / 'feature_k_sweep_quality_metrics.png',
        FIG_DIR / 'dtw_k_sweep_silhouette.png',
        FIG_DIR / 'genre_by_feature_cluster_counts.png',
        FIG_DIR / 'genre_by_feature_cluster_proportions.png',
        FIG_DIR / 'cluster_feature_signature_heatmap_top12.png',
        FIG_DIR / 'cluster_method_contingency_heatmap.png',
        FIG_DIR / 'feature_cluster_member_trajectories_ma5.png',
        FIG_DIR / 'cluster_centroid_trajectories_raw.png',
        FIG_DIR / 'cluster_centroid_trajectories_ma5.png',
        FIG_DIR / 'dtw_distance_heatmap.png',
        FIG_DIR / 'dtw_dendrogram.png',
        FIG_DIR / 'cluster_size_comparison.png',
        OUT_ROOT / 'insights.md',
        OUT_ROOT / 'cluster_report.md',
    ]
    return files


In [ ]:
# Load indep labels and validate integrity
meta = pd.read_csv(METADATA_PATH)
required_cols = {'id', 'title', 'processed_dir'}
if not required_cols.issubset(meta.columns):
    raise ValueError(f'metadata missing required columns: {required_cols - set(meta.columns)}')

meta = meta.sort_values('id').reset_index(drop=True)

integrity_rows = []
payloads = []

for r in meta.itertuples(index=False):
    book_id = int(r.id)
    title = str(r.title)
    processed_dir = str(r.processed_dir)
    base = PROCESSED_DIR / processed_dir

    label_path = base / LABEL_FILE
    emb_path = base / EMB_FILE

    if not label_path.exists():
        raise FileNotFoundError(f'Missing indep label file: {label_path}')
    if not emb_path.exists():
        raise FileNotFoundError(f'Missing embeddings file: {emb_path}')

    y_raw = np.load(label_path)
    y = normalize_label_shape(y_raw, label_path).astype(np.float64)

    emb_shape = np.load(emb_path, mmap_mode='r').shape
    if len(emb_shape) != 2:
        raise ValueError(f'Expected 2D embeddings at {emb_path}, got {emb_shape}')
    T_emb = int(emb_shape[0])
    T = int(len(y))

    int_like = bool(np.allclose(y, np.round(y)))
    in_range = bool(np.all((y >= 0) & (y <= 4)))
    aligned = bool(T == T_emb)

    integrity_rows.append({
        'check': f'book_{book_id}_indep_alignment',
        'expected': 'len==emb_T, integer-like labels in [0,4]',
        'actual': f'len={T}, emb_T={T_emb}, min={float(np.min(y)):.3f}, max={float(np.max(y)):.3f}, int_like={int_like}',
        'pass': bool(aligned and in_range and int_like),
    })

    if not aligned:
        raise ValueError(f'Length mismatch for {label_path}: label={T}, emb_T={T_emb}')
    if not in_range:
        raise ValueError(f'Out-of-range labels for {label_path}')
    if not int_like:
        raise ValueError(f'Non-integer-like labels for {label_path}')

    payloads.append({
        'book_id': book_id,
        'title': title,
        'processed_dir': processed_dir,
        'T': T,
        'y': y,
        'y_ma5': moving_average_1d(y, MA_WINDOW),
        'y_resampled': resample_to_len(y, L_DTW),
        'y_ma5_resampled': resample_to_len(moving_average_1d(y, MA_WINDOW), L_DTW),
    })

if len(payloads) != 20:
    raise RuntimeError(f'Expected 20 books, found {len(payloads)}')

print('Loaded books:', len(payloads))
print('Sample:', [(p['book_id'], p['processed_dir'], p['T']) for p in payloads[:5]])


In [ ]:
# Feature extraction + zscore + correlation figure
feature_rows = []
for p in payloads:
    feats = compute_features(p['y'], ma_window=MA_WINDOW)
    feature_rows.append({
        'book_id': int(p['book_id']),
        'title': p['title'],
        'processed_dir': p['processed_dir'],
        **feats,
    })

features_df = pd.DataFrame(feature_rows).sort_values('book_id').reset_index(drop=True)
id_cols = ['book_id', 'title', 'processed_dir']
numeric_cols = [c for c in features_df.columns if c not in id_cols]

means = features_df[numeric_cols].mean(axis=0)
stds = features_df[numeric_cols].std(axis=0, ddof=0).replace(0.0, 1.0)
z_df = features_df.copy()
z_df[numeric_cols] = (features_df[numeric_cols] - means) / stds

features_df.to_csv(TABLE_DIR / 'indep_book_features.csv', index=False)
z_df.to_csv(TABLE_DIR / 'indep_book_features_zscore.csv', index=False)

# Integrity checks for feature matrix
integrity_rows.extend([
    {'check': 'feature_row_count', 'expected': 20, 'actual': len(features_df), 'pass': len(features_df) == 20},
    {'check': 'feature_no_nan', 'expected': True, 'actual': bool(np.isfinite(features_df[numeric_cols].to_numpy()).all()), 'pass': bool(np.isfinite(features_df[numeric_cols].to_numpy()).all())},
    {'check': 'feature_no_inf_zscore', 'expected': True, 'actual': bool(np.isfinite(z_df[numeric_cols].to_numpy()).all()), 'pass': bool(np.isfinite(z_df[numeric_cols].to_numpy()).all())},
])

# Feature correlation heatmap
corr = features_df[numeric_cols].corr(numeric_only=True).to_numpy()
fig, ax = plt.subplots(figsize=(16, 13))
im = ax.imshow(corr, cmap='coolwarm', vmin=-1, vmax=1, aspect='auto')
ax.set_xticks(np.arange(len(numeric_cols)))
ax.set_xticklabels(numeric_cols, rotation=90, fontsize=8)
ax.set_yticks(np.arange(len(numeric_cols)))
ax.set_yticklabels(numeric_cols, fontsize=8)
ax.set_title('Feature correlation heatmap (indep_winsize_5)')
fig.colorbar(im, ax=ax, fraction=0.025, pad=0.01)
fig.tight_layout()
fig.savefig(FIG_DIR / 'feature_correlation_heatmap.png', dpi=180, bbox_inches='tight')
plt.close(fig)

print('Saved feature tables and correlation figure.')


In [ ]:
# Feature-branch clustering sweep (kmeans + ward) and selection
Xz = z_df[numeric_cols].to_numpy(dtype=np.float64)
book_ids = z_df['book_id'].to_numpy(dtype=int)

quality_rows = []
feature_assign_cache = {}

# KMeans elbow diagnostics (k=1..10)
elbow_rows = []
prev_inertia = None
for k in range(1, 11):
    km = KMeans(n_clusters=k, random_state=SEED, n_init=50)
    km.fit(Xz)
    inertia = float(km.inertia_)
    if prev_inertia is None:
        delta = np.nan
        pct_drop = np.nan
    else:
        delta = prev_inertia - inertia
        pct_drop = (delta / prev_inertia * 100.0) if prev_inertia > EPS else np.nan
    elbow_rows.append({
        'k': int(k),
        'inertia': inertia,
        'delta_inertia': delta,
        'pct_drop_from_prev': pct_drop,
    })
    prev_inertia = inertia

elbow_df = pd.DataFrame(elbow_rows)
elbow_df.to_csv(TABLE_DIR / 'kmeans_elbow_curve.csv', index=False)

# Feature clustering sweep (k=2..6)
for method in ['kmeans', 'ward']:
    for k in K_RANGE:
        if method == 'kmeans':
            model = KMeans(n_clusters=k, random_state=SEED, n_init=50)
        else:
            model = AgglomerativeClustering(n_clusters=k, linkage='ward')

        labels0 = model.fit_predict(Xz)
        feature_assign_cache[(method, k)] = labels0

        sil = float(silhouette_score(Xz, labels0, metric='euclidean'))
        db = float(davies_bouldin_score(Xz, labels0))
        ch = float(calinski_harabasz_score(Xz, labels0))

        stability = np.nan
        if method == 'kmeans':
            run_labels = []
            for s in range(N_STABILITY_SEEDS):
                km = KMeans(n_clusters=k, random_state=s, n_init=20)
                run_labels.append(km.fit_predict(Xz))
            aris = []
            for a, b in combinations(range(N_STABILITY_SEEDS), 2):
                aris.append(adjusted_rand_score(run_labels[a], run_labels[b]))
            stability = float(np.mean(aris)) if len(aris) else np.nan

        quality_rows.append({
            'branch': 'feature',
            'method': method,
            'k': int(k),
            'silhouette': sil,
            'davies_bouldin': db,
            'calinski_harabasz': ch,
            'kmeans_stability_ari': stability,
        })

feat_quality = pd.DataFrame([r for r in quality_rows if r['branch'] == 'feature']).copy()
feat_quality['stability_rank'] = feat_quality['kmeans_stability_ari'].fillna(-1e9)
feat_quality = feat_quality.sort_values(['silhouette', 'stability_rank', 'davies_bouldin'], ascending=[False, False, True]).reset_index(drop=True)

best_feat = feat_quality.iloc[0]
selected_feature_method = str(best_feat['method'])
selected_feature_k = int(best_feat['k'])
feature_labels0 = feature_assign_cache[(selected_feature_method, selected_feature_k)]
feature_labels = feature_labels0.astype(int) + 1

feature_cluster_ids = sorted(np.unique(feature_labels))
feature_cluster_sizes = {int(c): int(np.sum(feature_labels == c)) for c in feature_cluster_ids}
feature_palette = plt.cm.tab10(np.linspace(0, 1, 10))
feature_color_map = {int(c): feature_palette[i % len(feature_palette)] for i, c in enumerate(feature_cluster_ids)}

integrity_rows.append({
    'check': 'selected_feature_k_in_range',
    'expected': '[2,6]',
    'actual': selected_feature_k,
    'pass': 2 <= selected_feature_k <= 6,
})

# Elbow integrity + figure
elbow_monotonic = bool(np.all(np.diff(elbow_df['inertia'].to_numpy(dtype=np.float64)) <= 1e-9))
inertia_selected = float(elbow_df.loc[elbow_df['k'] == selected_feature_k, 'inertia'].iloc[0])

fig, ax = plt.subplots(figsize=(8.5, 5.5))
ax.plot(elbow_df['k'], elbow_df['inertia'], marker='o', linewidth=2.0, color='#1f77b4')
ax.scatter([selected_feature_k], [inertia_selected], s=70, color='#d62728', zorder=5, label=f'selected k={selected_feature_k}')
ax.axvline(selected_feature_k, linestyle='--', color='#d62728', alpha=0.7)
ax.set_title('KMeans elbow curve (diagnostic only)')
ax.set_xlabel('k')
ax.set_ylabel('Inertia')
ax.grid(alpha=0.25)
ax.legend(loc='upper right')
fig.tight_layout()
fig.savefig(FIG_DIR / 'feature_elbow_kmeans_inertia.png', dpi=180, bbox_inches='tight')
plt.close(fig)

# Feature PCA scatter with robust legend handling
pca = PCA(n_components=2, random_state=SEED)
X2 = pca.fit_transform(Xz)
fig, ax = plt.subplots(figsize=(11, 8))
legend_handles = []
legend_labels = []
for c in feature_cluster_ids:
    idx = np.where(feature_labels == c)[0]
    color = feature_color_map[int(c)]
    sc = ax.scatter(
        X2[idx, 0],
        X2[idx, 1],
        s=85,
        alpha=0.9,
        color=color,
        edgecolor='black',
        linewidth=0.35,
    )
    legend_handles.append(sc)
    legend_labels.append(f'Cluster {int(c)} (n={feature_cluster_sizes[int(c)]})')
    for j in idx:
        ax.text(X2[j, 0], X2[j, 1], str(int(book_ids[j])), fontsize=8, alpha=0.82)

ax.set_title(f'Feature PCA scatter (method={selected_feature_method}, k={selected_feature_k})')
ax.set_xlabel('PC1')
ax.set_ylabel('PC2')
ax.grid(alpha=0.2)
legend = ax.legend(
    handles=legend_handles,
    labels=legend_labels,
    title='Feature clusters',
    loc='upper left',
    bbox_to_anchor=(1.02, 1.0),
    borderaxespad=0.0,
    frameon=True,
)
fig.tight_layout(rect=[0, 0, 0.80, 1])
fig.savefig(FIG_DIR / 'feature_pca_scatter_feature_clusters.png', dpi=180, bbox_inches='tight')
plt.close(fig)

legend_actual = len(legend.texts) if legend is not None else 0
legend_df = pd.DataFrame([
    {
        'figure': 'feature_pca_scatter_feature_clusters.png',
        'expected_entries': int(len(feature_cluster_ids)),
        'actual_entries': int(legend_actual),
        'pass': bool(legend_actual == len(feature_cluster_ids)),
        'labels': ' | '.join(legend_labels),
    }
])
legend_df.to_csv(TABLE_DIR / 'figure_legend_checks.csv', index=False)

integrity_rows.extend([
    {'check': 'elbow_k_coverage', 'expected': '1..10', 'actual': f"{int(elbow_df['k'].min())}..{int(elbow_df['k'].max())}", 'pass': int(elbow_df['k'].min()) == 1 and int(elbow_df['k'].max()) == 10},
    {'check': 'elbow_inertia_nonincreasing', 'expected': True, 'actual': elbow_monotonic, 'pass': elbow_monotonic},
    {'check': 'feature_pca_legend_entries_match_clusters', 'expected': int(len(feature_cluster_ids)), 'actual': int(legend_actual), 'pass': bool(legend_actual == len(feature_cluster_ids))},
])

print('Selected feature clustering:', selected_feature_method, selected_feature_k)


In [ ]:
# DTW branch + method agreement + cluster tables + representative/profile tables

# Build DTW distance matrix from resampled trajectories
Y_res = np.vstack([p['y_resampled'] for p in payloads])
N = Y_res.shape[0]
D = np.zeros((N, N), dtype=np.float64)
for i in range(N):
    for j in range(i + 1, N):
        d = dtw_distance(Y_res[i], Y_res[j])
        D[i, j] = d
        D[j, i] = d

# DTW clustering sweep
DTW_METHOD = 'average'
dtw_assign_cache = {}
for k in K_RANGE:
    model = AgglomerativeClustering(n_clusters=k, metric='precomputed', linkage=DTW_METHOD)
    labels0 = model.fit_predict(D)
    dtw_assign_cache[k] = labels0
    sil = float(silhouette_score(D, labels0, metric='precomputed'))
    quality_rows.append({
        'branch': 'dtw',
        'method': DTW_METHOD,
        'k': int(k),
        'silhouette': sil,
        'davies_bouldin': np.nan,
        'calinski_harabasz': np.nan,
        'kmeans_stability_ari': np.nan,
    })

quality_df = pd.DataFrame(quality_rows).sort_values(['branch', 'method', 'k']).reset_index(drop=True)
dtw_quality = quality_df[quality_df['branch'] == 'dtw'].sort_values('silhouette', ascending=False).reset_index(drop=True)
best_dtw = dtw_quality.iloc[0]
selected_dtw_k = int(best_dtw['k'])
dtw_labels0 = dtw_assign_cache[selected_dtw_k]
dtw_labels = dtw_labels0.astype(int) + 1

dtw_cluster_ids = sorted(np.unique(dtw_labels))
dtw_cluster_sizes = {int(c): int(np.sum(dtw_labels == c)) for c in dtw_cluster_ids}
dtw_palette = plt.cm.Set2(np.linspace(0, 1, max(3, len(dtw_cluster_ids))))
dtw_color_map = {int(c): dtw_palette[i % len(dtw_palette)] for i, c in enumerate(dtw_cluster_ids)}

integrity_rows.append({
    'check': 'selected_dtw_k_in_range',
    'expected': '[2,6]',
    'actual': selected_dtw_k,
    'pass': 2 <= selected_dtw_k <= 6,
})

# Assignments tables
feature_assign_df = z_df[id_cols].copy()
feature_assign_df['method'] = selected_feature_method
feature_assign_df['k'] = selected_feature_k
feature_assign_df['cluster'] = feature_labels
feature_assign_df = feature_assign_df.sort_values('book_id').reset_index(drop=True)

dtw_assign_df = z_df[id_cols].copy()
dtw_assign_df['method'] = DTW_METHOD
dtw_assign_df['k'] = selected_dtw_k
dtw_assign_df['cluster'] = dtw_labels
dtw_assign_df = dtw_assign_df.sort_values('book_id').reset_index(drop=True)

# Agreement table with metrics + contingency
ari = float(adjusted_rand_score(feature_labels, dtw_labels))
nmi = float(normalized_mutual_info_score(feature_labels, dtw_labels))
ctab = pd.crosstab(feature_labels, dtw_labels)

agree_rows = [
    {'row_type': 'metric', 'metric': 'ari', 'value': ari, 'feature_cluster': np.nan, 'dtw_cluster': np.nan, 'count': np.nan},
    {'row_type': 'metric', 'metric': 'nmi', 'value': nmi, 'feature_cluster': np.nan, 'dtw_cluster': np.nan, 'count': np.nan},
]
for fc in ctab.index:
    for dc in ctab.columns:
        agree_rows.append({
            'row_type': 'contingency',
            'metric': 'count',
            'value': np.nan,
            'feature_cluster': int(fc),
            'dtw_cluster': int(dc),
            'count': int(ctab.loc[fc, dc]),
        })
agreement_df = pd.DataFrame(agree_rows)

# Quality metric visualizations
feature_quality = quality_df[quality_df['branch'] == 'feature'].copy()

fig, axes = plt.subplots(2, 2, figsize=(14, 10))
for method in ['kmeans', 'ward']:
    s = feature_quality[feature_quality['method'] == method].sort_values('k')
    axes[0, 0].plot(s['k'], s['silhouette'], marker='o', linewidth=2, label=method)
axes[0, 0].set_title('Silhouette vs k (feature branch)')
axes[0, 0].set_xlabel('k')
axes[0, 0].set_ylabel('Silhouette')
axes[0, 0].grid(alpha=0.25)
axes[0, 0].legend()

for method in ['kmeans', 'ward']:
    s = feature_quality[feature_quality['method'] == method].sort_values('k')
    axes[0, 1].plot(s['k'], s['davies_bouldin'], marker='o', linewidth=2, label=method)
axes[0, 1].set_title('Davies-Bouldin vs k (lower is better)')
axes[0, 1].set_xlabel('k')
axes[0, 1].set_ylabel('Davies-Bouldin')
axes[0, 1].grid(alpha=0.25)

for method in ['kmeans', 'ward']:
    s = feature_quality[feature_quality['method'] == method].sort_values('k')
    axes[1, 0].plot(s['k'], s['calinski_harabasz'], marker='o', linewidth=2, label=method)
axes[1, 0].set_title('Calinski-Harabasz vs k (higher is better)')
axes[1, 0].set_xlabel('k')
axes[1, 0].set_ylabel('Calinski-Harabasz')
axes[1, 0].grid(alpha=0.25)

km_stability = feature_quality[feature_quality['method'] == 'kmeans'].sort_values('k')
axes[1, 1].plot(km_stability['k'], km_stability['kmeans_stability_ari'], marker='o', linewidth=2, color='#9467bd')
axes[1, 1].set_title('KMeans stability ARI vs k')
axes[1, 1].set_xlabel('k')
axes[1, 1].set_ylabel('Mean pairwise ARI')
axes[1, 1].grid(alpha=0.25)

fig.suptitle('Feature branch quality diagnostics')
fig.tight_layout()
fig.savefig(FIG_DIR / 'feature_k_sweep_quality_metrics.png', dpi=180, bbox_inches='tight')
plt.close(fig)

fig, ax = plt.subplots(figsize=(8.5, 5.5))
s = dtw_quality.sort_values('k')
ax.plot(s['k'], s['silhouette'], marker='o', linewidth=2, color='#2ca02c')
ax.scatter([selected_dtw_k], [float(s[s['k'] == selected_dtw_k]['silhouette'].iloc[0])], color='#d62728', s=70, zorder=5, label=f'selected k={selected_dtw_k}')
ax.axvline(selected_dtw_k, linestyle='--', color='#d62728', alpha=0.7)
ax.set_title('DTW branch silhouette vs k')
ax.set_xlabel('k')
ax.set_ylabel('Silhouette (precomputed DTW distance)')
ax.grid(alpha=0.25)
ax.legend(loc='upper right')
fig.tight_layout()
fig.savefig(FIG_DIR / 'dtw_k_sweep_silhouette.png', dpi=180, bbox_inches='tight')
plt.close(fig)

# Genre by feature cluster analysis
genre_map = (
    meta[['id', 'genre_primary']]
    .drop_duplicates(subset=['id'])
    .rename(columns={'id': 'book_id'})
)
feature_genre_df = feature_assign_df.merge(genre_map, on='book_id', how='left')
missing_genre = int(feature_genre_df['genre_primary'].isna().sum())
feature_genre_df['genre_primary'] = feature_genre_df['genre_primary'].fillna('Unknown')

genre_count_df = pd.crosstab(feature_genre_df['cluster'], feature_genre_df['genre_primary']).sort_index()
genre_prop_df = genre_count_df.div(genre_count_df.sum(axis=1), axis=0).fillna(0.0)

genre_count_df.to_csv(TABLE_DIR / 'genre_by_feature_cluster_counts.csv', index_label='cluster')
genre_prop_df.to_csv(TABLE_DIR / 'genre_by_feature_cluster_proportions.csv', index_label='cluster')

fig, ax = plt.subplots(figsize=(12, 6))
genre_count_df.plot(kind='bar', stacked=True, ax=ax, colormap='tab20')
ax.set_title('Genre composition by feature cluster (counts)')
ax.set_xlabel('Feature cluster')
ax.set_ylabel('Books')
ax.grid(axis='y', alpha=0.25)
ax.legend(title='genre_primary', bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0)
fig.tight_layout(rect=[0, 0, 0.80, 1])
fig.savefig(FIG_DIR / 'genre_by_feature_cluster_counts.png', dpi=180, bbox_inches='tight')
plt.close(fig)

fig, ax = plt.subplots(figsize=(11, 5.5))
mat = genre_prop_df.to_numpy(dtype=np.float64)
im = ax.imshow(mat, cmap='YlGnBu', aspect='auto', vmin=0.0, vmax=max(0.25, float(np.max(mat))))
ax.set_xticks(np.arange(genre_prop_df.shape[1]))
ax.set_xticklabels(genre_prop_df.columns, rotation=45, ha='right')
ax.set_yticks(np.arange(genre_prop_df.shape[0]))
ax.set_yticklabels([str(int(i)) for i in genre_prop_df.index])
ax.set_xlabel('genre_primary')
ax.set_ylabel('Feature cluster')
ax.set_title('Genre composition by feature cluster (row-normalized)')
for r in range(mat.shape[0]):
    for c in range(mat.shape[1]):
        if mat[r, c] > 0:
            ax.text(c, r, f'{mat[r, c]:.2f}', ha='center', va='center', fontsize=8, color='black')
fig.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
fig.tight_layout()
fig.savefig(FIG_DIR / 'genre_by_feature_cluster_proportions.png', dpi=180, bbox_inches='tight')
plt.close(fig)

# Profile summary table
profile_rows = []
rep_rows = []

Xz_df = z_df.copy()
Xz_df['cluster_feature'] = feature_labels
Xz_df['cluster_dtw'] = dtw_labels

# Feature-branch representatives (centroid nearest + high/low volatility)
for c in feature_cluster_ids:
    idx = np.where(feature_labels == c)[0]
    Xc = Xz[idx]
    centroid = Xc.mean(axis=0)
    dists = np.linalg.norm(Xc - centroid, axis=1)
    medoid_local = idx[int(np.argmin(dists))]

    raw_sub = features_df.iloc[idx].copy()
    high_idx = raw_sub['std_diff'].idxmax()
    low_idx = raw_sub['std_diff'].idxmin()

    for role, ridx, sval, sname in [
        ('centroid_medoid', medoid_local, float(np.min(dists)), 'euclidean_to_centroid'),
        ('high_volatility', int(high_idx), float(features_df.loc[high_idx, 'std_diff']), 'std_diff'),
        ('low_volatility', int(low_idx), float(features_df.loc[low_idx, 'std_diff']), 'std_diff'),
    ]:
        rep_rows.append({
            'branch': 'feature',
            'cluster': int(c),
            'role': role,
            'book_id': int(features_df.loc[ridx, 'book_id']),
            'title': str(features_df.loc[ridx, 'title']),
            'processed_dir': str(features_df.loc[ridx, 'processed_dir']),
            'score_name': sname,
            'score_value': float(sval),
        })

# DTW-branch representatives (medoid via mean intra-cluster DTW + high/low volatility)
for c in dtw_cluster_ids:
    idx = np.where(dtw_labels == c)[0]
    Dc = D[np.ix_(idx, idx)]
    mean_d = Dc.mean(axis=1)
    medoid_local = idx[int(np.argmin(mean_d))]

    raw_sub = features_df.iloc[idx].copy()
    high_idx = raw_sub['std_diff'].idxmax()
    low_idx = raw_sub['std_diff'].idxmin()

    for role, ridx, sval, sname in [
        ('cluster_medoid', medoid_local, float(np.min(mean_d)), 'mean_intra_cluster_dtw'),
        ('high_volatility', int(high_idx), float(features_df.loc[high_idx, 'std_diff']), 'std_diff'),
        ('low_volatility', int(low_idx), float(features_df.loc[low_idx, 'std_diff']), 'std_diff'),
    ]:
        rep_rows.append({
            'branch': 'dtw',
            'cluster': int(c),
            'role': role,
            'book_id': int(features_df.loc[ridx, 'book_id']),
            'title': str(features_df.loc[ridx, 'title']),
            'processed_dir': str(features_df.loc[ridx, 'processed_dir']),
            'score_name': sname,
            'score_value': float(sval),
        })

# Profile rows (feature + dtw branches)
for branch_name, cluster_col in [('feature', 'cluster_feature'), ('dtw', 'cluster_dtw')]:
    for c in sorted(Xz_df[cluster_col].unique()):
        mask = Xz_df[cluster_col] == c
        n_c = int(mask.sum())
        for feat in numeric_cols:
            mean_raw = float(features_df.loc[mask, feat].mean())
            med_raw = float(features_df.loc[mask, feat].median())
            mean_z = float(z_df.loc[mask, feat].mean())
            profile_rows.append({
                'branch': branch_name,
                'cluster': int(c),
                'cluster_size': n_c,
                'feature': feat,
                'mean_raw': mean_raw,
                'median_raw': med_raw,
                'mean_z': mean_z,
                'delta_z_vs_corpus': mean_z,
            })

profile_df = pd.DataFrame(profile_rows)
profile_df['abs_delta_z'] = profile_df['delta_z_vs_corpus'].abs()
profile_df['rank_abs_delta'] = (
    profile_df.sort_values(['branch', 'cluster', 'abs_delta_z'], ascending=[True, True, False])
    .groupby(['branch', 'cluster'])
    .cumcount() + 1
)

representatives_df = pd.DataFrame(rep_rows).sort_values(['branch', 'cluster', 'role']).reset_index(drop=True)

# Feature signature heatmap/table (top 12 by max absolute delta)
feature_profile = profile_df[profile_df['branch'] == 'feature'].copy()
feature_max_abs = (
    feature_profile.groupby('feature', as_index=False)['abs_delta_z']
    .max()
    .sort_values('abs_delta_z', ascending=False)
)
top_signature_features = feature_max_abs.head(12)['feature'].tolist()
feature_rank_map = {f: i + 1 for i, f in enumerate(top_signature_features)}
feature_signature_df = feature_profile[feature_profile['feature'].isin(top_signature_features)].copy()
feature_signature_df['global_feature_rank'] = feature_signature_df['feature'].map(feature_rank_map)
feature_signature_df = feature_signature_df.sort_values(['global_feature_rank', 'cluster']).reset_index(drop=True)
feature_signature_df.to_csv(TABLE_DIR / 'feature_cluster_signature_top_features.csv', index=False)

sig_pivot = (
    feature_signature_df
    .pivot_table(index='cluster', columns='feature', values='delta_z_vs_corpus', aggfunc='mean')
    .reindex(sorted(feature_signature_df['cluster'].unique()), axis=0)
)
sig_pivot = sig_pivot[top_signature_features]

fig, ax = plt.subplots(figsize=(14, 5.5))
mat = sig_pivot.to_numpy(dtype=np.float64)
vmax = float(np.max(np.abs(mat))) if np.isfinite(mat).all() else 1.0
vmax = max(vmax, 0.1)
im = ax.imshow(mat, cmap='coolwarm', aspect='auto', vmin=-vmax, vmax=vmax)
ax.set_xticks(np.arange(sig_pivot.shape[1]))
ax.set_xticklabels(sig_pivot.columns, rotation=45, ha='right')
ax.set_yticks(np.arange(sig_pivot.shape[0]))
ax.set_yticklabels([str(int(i)) for i in sig_pivot.index])
ax.set_xlabel('Feature')
ax.set_ylabel('Feature cluster')
ax.set_title('Top-12 feature signatures by cluster (delta z vs corpus)')
for r in range(mat.shape[0]):
    for c in range(mat.shape[1]):
        ax.text(c, r, f'{mat[r, c]:+.2f}', ha='center', va='center', fontsize=8)
fig.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
fig.tight_layout()
fig.savefig(FIG_DIR / 'cluster_feature_signature_heatmap_top12.png', dpi=180, bbox_inches='tight')
plt.close(fig)

# Contingency heatmap (feature vs DTW)
fig, ax = plt.subplots(figsize=(7, 5.5))
ctab_mat = ctab.to_numpy(dtype=np.float64)
im = ax.imshow(ctab_mat, cmap='Blues', aspect='auto')
ax.set_xticks(np.arange(len(ctab.columns)))
ax.set_xticklabels([f'DTW {int(c)}' for c in ctab.columns])
ax.set_yticks(np.arange(len(ctab.index)))
ax.set_yticklabels([f'Feat {int(c)}' for c in ctab.index])
ax.set_title('Feature vs DTW cluster contingency')
ax.set_xlabel('DTW cluster')
ax.set_ylabel('Feature cluster')
for r in range(ctab_mat.shape[0]):
    for c in range(ctab_mat.shape[1]):
        ax.text(c, r, f'{int(ctab_mat[r, c])}', ha='center', va='center', fontsize=10)
fig.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
fig.tight_layout()
fig.savefig(FIG_DIR / 'cluster_method_contingency_heatmap.png', dpi=180, bbox_inches='tight')
plt.close(fig)

# Save tables
quality_df.to_csv(TABLE_DIR / 'cluster_quality_by_method.csv', index=False)
feature_assign_df.to_csv(TABLE_DIR / 'cluster_assignments_feature.csv', index=False)
dtw_assign_df.to_csv(TABLE_DIR / 'cluster_assignments_dtw.csv', index=False)
profile_df.to_csv(TABLE_DIR / 'cluster_profile_summary.csv', index=False)
representatives_df.to_csv(TABLE_DIR / 'cluster_representatives.csv', index=False)
agreement_df.to_csv(TABLE_DIR / 'cluster_method_agreement.csv', index=False)

# Integrity checks for cluster assignments + quality table + genre checks
row_sum_dev = float(np.max(np.abs(genre_prop_df.sum(axis=1).to_numpy(dtype=np.float64) - 1.0))) if len(genre_prop_df) else 0.0
integrity_rows.extend([
    {'check': 'cluster_assignments_feature_rows', 'expected': 20, 'actual': len(feature_assign_df), 'pass': len(feature_assign_df) == 20},
    {'check': 'cluster_assignments_dtw_rows', 'expected': 20, 'actual': len(dtw_assign_df), 'pass': len(dtw_assign_df) == 20},
    {'check': 'cluster_quality_rows', 'expected': len(K_RANGE) * 3, 'actual': len(quality_df), 'pass': len(quality_df) == len(K_RANGE) * 3},
    {'check': 'agreement_ari_range', 'expected': '[0,1]', 'actual': ari, 'pass': 0.0 <= ari <= 1.0},
    {'check': 'agreement_nmi_range', 'expected': '[0,1]', 'actual': nmi, 'pass': 0.0 <= nmi <= 1.0},
    {'check': 'genre_join_missing_count', 'expected': 0, 'actual': missing_genre, 'pass': missing_genre == 0},
    {'check': 'genre_cluster_count_sum', 'expected': len(feature_assign_df), 'actual': int(genre_count_df.to_numpy().sum()), 'pass': int(genre_count_df.to_numpy().sum()) == len(feature_assign_df)},
    {'check': 'genre_proportion_rowsum_max_deviation', 'expected': '<=1e-6', 'actual': row_sum_dev, 'pass': row_sum_dev <= 1e-6},
])

print('Selected feature:', selected_feature_method, selected_feature_k)
print('Selected DTW:', selected_dtw_k)
print('ARI/NMI:', ari, nmi)


In [ ]:
# Figures: DTW + centroids + cluster sizes + member panels

# DTW distance heatmap (ordered by dtw cluster)
order = np.argsort(dtw_labels)
D_ord = D[np.ix_(order, order)]

fig, ax = plt.subplots(figsize=(9, 8))
im = ax.imshow(D_ord, cmap='viridis', aspect='auto')
ax.set_title('DTW distance heatmap (ordered by DTW clusters)')
ax.set_xlabel('Books')
ax.set_ylabel('Books')
fig.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
fig.tight_layout()
fig.savefig(FIG_DIR / 'dtw_distance_heatmap.png', dpi=180, bbox_inches='tight')
plt.close(fig)

# DTW dendrogram
condensed = squareform(D, checks=False)
Z = linkage(condensed, method=DTW_METHOD)
fig, ax = plt.subplots(figsize=(12, 6))
dendrogram(Z, labels=[str(int(b)) for b in book_ids], leaf_rotation=90, ax=ax)
ax.set_title('DTW hierarchical dendrogram (average linkage)')
ax.set_xlabel('Book ID')
ax.set_ylabel('Distance')
fig.tight_layout()
fig.savefig(FIG_DIR / 'dtw_dendrogram.png', dpi=180, bbox_inches='tight')
plt.close(fig)

# Cluster centroid trajectories (raw and MA5), feature vs dtw
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(16, 5.5), sharey=True)
for ax, branch, labels, color_map in [
    (axes[0], 'feature', feature_labels, feature_color_map),
    (axes[1], 'dtw', dtw_labels, dtw_color_map),
]:
    for c in sorted(np.unique(labels)):
        idx = np.where(labels == c)[0]
        Ys = np.vstack([payloads[i]['y_resampled'] for i in idx])
        centroid = Ys.mean(axis=0)
        color = color_map[int(c)]
        for row in Ys:
            ax.plot(np.linspace(0, 1, L_DTW), row, color=color, alpha=0.10, linewidth=0.8)
        ax.plot(np.linspace(0, 1, L_DTW), centroid, color=color, linewidth=2.2, label=f'Cluster {int(c)} (n={len(idx)})')
    ax.set_title(f'{branch} clusters (raw)')
    ax.set_xlabel('Normalized position')
    ax.grid(alpha=0.2)
axes[0].set_ylabel('Excitement')
axes[0].legend(loc='upper right', fontsize=8)
axes[1].legend(loc='upper right', fontsize=8)
fig.tight_layout()
fig.savefig(FIG_DIR / 'cluster_centroid_trajectories_raw.png', dpi=180, bbox_inches='tight')
plt.close(fig)

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(16, 5.5), sharey=True)
for ax, branch, labels, color_map in [
    (axes[0], 'feature', feature_labels, feature_color_map),
    (axes[1], 'dtw', dtw_labels, dtw_color_map),
]:
    for c in sorted(np.unique(labels)):
        idx = np.where(labels == c)[0]
        Ys = np.vstack([payloads[i]['y_ma5_resampled'] for i in idx])
        centroid = Ys.mean(axis=0)
        color = color_map[int(c)]
        for row in Ys:
            ax.plot(np.linspace(0, 1, L_DTW), row, color=color, alpha=0.10, linewidth=0.8)
        ax.plot(np.linspace(0, 1, L_DTW), centroid, color=color, linewidth=2.2, label=f'Cluster {int(c)} (n={len(idx)})')
    ax.set_title(f'{branch} clusters (MA5)')
    ax.set_xlabel('Normalized position')
    ax.grid(alpha=0.2)
axes[0].set_ylabel('Excitement')
axes[0].legend(loc='upper right', fontsize=8)
axes[1].legend(loc='upper right', fontsize=8)
fig.tight_layout()
fig.savefig(FIG_DIR / 'cluster_centroid_trajectories_ma5.png', dpi=180, bbox_inches='tight')
plt.close(fig)

# Member trajectories by feature cluster (MA5)
feature_clusters = sorted(np.unique(feature_labels))
n_clusters = len(feature_clusters)
ncols = 2
nrows = int(np.ceil(n_clusters / ncols))
fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(14, 3.8 * nrows), sharex=True, sharey=True)
axes = np.atleast_1d(axes).reshape(nrows, ncols)
for i, c in enumerate(feature_clusters):
    ax = axes[i // ncols, i % ncols]
    idx = np.where(feature_labels == c)[0]
    color = feature_color_map[int(c)]
    Ys = np.vstack([payloads[j]['y_ma5_resampled'] for j in idx])
    centroid = Ys.mean(axis=0)
    for row in Ys:
        ax.plot(np.linspace(0, 1, L_DTW), row, color=color, alpha=0.18, linewidth=0.9)
    ax.plot(np.linspace(0, 1, L_DTW), centroid, color=color, linewidth=2.4)
    ax.set_title(f'Feature Cluster {int(c)} (n={len(idx)})')
    ax.grid(alpha=0.2)
for j in range(n_clusters, nrows * ncols):
    axes[j // ncols, j % ncols].axis('off')
for ax in axes[-1, :]:
    ax.set_xlabel('Normalized position')
for r in range(nrows):
    axes[r, 0].set_ylabel('MA5 excitement')
fig.suptitle('Feature cluster member trajectories (MA5)')
fig.tight_layout()
fig.savefig(FIG_DIR / 'feature_cluster_member_trajectories_ma5.png', dpi=180, bbox_inches='tight')
plt.close(fig)

# Cluster size comparison
feat_counts = pd.Series(feature_labels).value_counts().sort_index()
dtw_counts = pd.Series(dtw_labels).value_counts().sort_index()
all_clusters = sorted(set(feat_counts.index.tolist()) | set(dtw_counts.index.tolist()))
feat_vals = np.array([int(feat_counts.get(c, 0)) for c in all_clusters])
dtw_vals = np.array([int(dtw_counts.get(c, 0)) for c in all_clusters])

x = np.arange(len(all_clusters))
bw = 0.35
fig, ax = plt.subplots(figsize=(10, 5))
ax.bar(x - bw/2, feat_vals, width=bw, label=f'feature ({selected_feature_method}, k={selected_feature_k})')
ax.bar(x + bw/2, dtw_vals, width=bw, label=f'dtw (k={selected_dtw_k})')
ax.set_xticks(x)
ax.set_xticklabels([str(int(c)) for c in all_clusters])
ax.set_xlabel('Cluster id')
ax.set_ylabel('Books')
ax.set_title('Cluster size comparison: feature vs DTW')
ax.grid(axis='y', alpha=0.25)
ax.legend()
fig.tight_layout()
fig.savefig(FIG_DIR / 'cluster_size_comparison.png', dpi=180, bbox_inches='tight')
plt.close(fig)

print('Saved DTW/centroid/size/member figures.')


In [ ]:
# Insights markdown + cluster report + finalize integrity checks

# Build top distinguishing feature helper
feature_top = (
    profile_df[profile_df['branch'] == 'feature']
    .sort_values(['cluster', 'rank_abs_delta'])
)
dtw_top = (
    profile_df[profile_df['branch'] == 'dtw']
    .sort_values(['cluster', 'rank_abs_delta'])
)

rep_feature = representatives_df[(representatives_df['branch'] == 'feature') & (representatives_df['role'] == 'centroid_medoid')]
rep_dtw = representatives_df[(representatives_df['branch'] == 'dtw') & (representatives_df['role'] == 'cluster_medoid')]

# Existing concise insights.md
lines = []
lines.append('# indep_winsize_5 Clustering Insights (MA(W), current run W=5)')
lines.append('')
lines.append('## 1) Setup and Integrity')
lines.append(f'- Books analyzed: **{len(payloads)}**')
lines.append('- Input checks passed: label existence, shape normalization, range `[0,4]`, integer-like, and embedding length alignment.')
lines.append(f'- Feature set size: **{len(numeric_cols)}** numeric features per book.')
lines.append('')

feat_row = quality_df[(quality_df['branch']=='feature') & (quality_df['method']==selected_feature_method) & (quality_df['k']==selected_feature_k)].iloc[0]
dtw_row = quality_df[(quality_df['branch']=='dtw') & (quality_df['k']==selected_dtw_k)].iloc[0]

lines.append('## 2) Model Selection Results')
lines.append(f"- Feature branch selected: `{selected_feature_method}` with `k={selected_feature_k}` (silhouette={feat_row['silhouette']:.3f}, DB={feat_row['davies_bouldin']:.3f}, CH={feat_row['calinski_harabasz']:.1f}).")
if selected_feature_method == 'kmeans':
    lines.append(f"- KMeans stability (mean pairwise ARI across seeds): {feat_row['kmeans_stability_ari']:.3f}.")
lines.append(f"- DTW branch selected: `average-linkage` with `k={selected_dtw_k}` (silhouette={dtw_row['silhouette']:.3f}).")
lines.append('')

lines.append('## 3) Cross-Method Agreement')
lines.append(f'- Adjusted Rand Index (feature vs DTW): **{ari:.3f}**')
lines.append(f'- Normalized Mutual Information: **{nmi:.3f}**')
lines.append('- Agreement table: `outputs/excitement_indep_clustering/tables/cluster_method_agreement.csv`.')
lines.append('')

lines.append('## 4) Feature-Branch Cluster Archetypes')
for c in sorted(np.unique(feature_labels)):
    top_feats = feature_top[(feature_top['cluster']==c) & (feature_top['rank_abs_delta']<=3)]
    top_txt = '; '.join([f"{r.feature} (delta_z={r.delta_z_vs_corpus:+.2f})" for r in top_feats.itertuples(index=False)])
    rep = rep_feature[rep_feature['cluster']==c].iloc[0]
    csize = int(np.sum(feature_labels == c))
    lines.append(f"- Cluster {int(c)} (n={csize}): {top_txt}. Representative: {int(rep['book_id'])} | {rep['title']}.")
lines.append('')

lines.append('## 5) DTW-Branch Shape Archetypes')
for c in sorted(np.unique(dtw_labels)):
    top_feats = dtw_top[(dtw_top['cluster']==c) & (dtw_top['rank_abs_delta']<=3)]
    top_txt = '; '.join([f"{r.feature} (delta_z={r.delta_z_vs_corpus:+.2f})" for r in top_feats.itertuples(index=False)])
    rep = rep_dtw[rep_dtw['cluster']==c].iloc[0]
    csize = int(np.sum(dtw_labels == c))
    lines.append(f"- Cluster {int(c)} (n={csize}): {top_txt}. DTW medoid: {int(rep['book_id'])} | {rep['title']}.")
lines.append('')

lines.append('## 6) Practical Reading')
lines.append('- Use feature clusters as primary interpretable archetypes.')
lines.append('- Use DTW clusters as trajectory-shape validation; disagreement flags uncertain archetypes.')
lines.append('- MA5 centroid plots are better for coarse pacing patterns than raw per-chunk spikes.')
lines.append('')

lines.append('## 7) Next Steps')
lines.append('1. Add bootstrap stability over book-resampling for selected k.')
lines.append('2. Compare clustering using raw vs MA5-only feature subsets.')
lines.append('3. Add external validation against genre/twist metadata as weak labels.')
lines.append('')

lines.append('## Provenance')
lines.append('- Figures: `outputs/excitement_indep_clustering/figures/*.png`')
lines.append('- Tables: `outputs/excitement_indep_clustering/tables/*.csv`')

(OUT_ROOT / 'insights.md').write_text(chr(10).join(lines) + chr(10), encoding='utf-8')

# New extended cluster report with embedded figures
dtw_sweep = quality_df[quality_df['branch'] == 'dtw'].sort_values('k').reset_index(drop=True)
feature_sweep = quality_df[quality_df['branch'] == 'feature'].sort_values(['method', 'k']).reset_index(drop=True)

elbow_valid = elbow_df.dropna(subset=['delta_inertia']).copy()
if len(elbow_valid):
    elbow_best_idx = int(elbow_valid['delta_inertia'].idxmax())
    elbow_best_k = int(elbow_df.loc[elbow_best_idx, 'k'])
    elbow_best_drop = float(elbow_df.loc[elbow_best_idx, 'delta_inertia'])
else:
    elbow_best_k = selected_feature_k
    elbow_best_drop = float('nan')

genre_count_df = pd.read_csv(TABLE_DIR / 'genre_by_feature_cluster_counts.csv', index_col='cluster')
genre_prop_df = pd.read_csv(TABLE_DIR / 'genre_by_feature_cluster_proportions.csv', index_col='cluster')
feature_signature_df = pd.read_csv(TABLE_DIR / 'feature_cluster_signature_top_features.csv')

report_lines = []
report_lines.append('# indep_winsize_5 Clustering Report (Feature-Primary, MA(W) with W=5)')
report_lines.append('')
report_lines.append('## Executive Verdict')
report_lines.append(f"- The **feature branch** is primary for interpretation: selected `{selected_feature_method}` with `k={selected_feature_k}`.")
report_lines.append(f"- Feature quality at selected solution: silhouette `{feat_row['silhouette']:.3f}`, Davies-Bouldin `{feat_row['davies_bouldin']:.3f}`, Calinski-Harabasz `{feat_row['calinski_harabasz']:.2f}`.")
if selected_feature_method == 'kmeans':
    report_lines.append(f"- KMeans seed-stability is high (mean pairwise ARI `{feat_row['kmeans_stability_ari']:.3f}`).")
report_lines.append(f"- DTW validation selected `k={selected_dtw_k}` with silhouette `{dtw_row['silhouette']:.3f}`; agreement with feature clusters is limited (ARI `{ari:.3f}`, NMI `{nmi:.3f}`).")
report_lines.append('')

report_lines.append('## Model Selection Diagnostics')
report_lines.append(f"- Elbow diagnostics (`k=1..10`) show largest inertia drop at `k={elbow_best_k}` (drop `{elbow_best_drop:.3f}`).")
report_lines.append('- Final model-selection policy remains silhouette-first over `k=2..6` for comparability with prior runs.')
report_lines.append('- DTW branch is used as trajectory-shape validation rather than the primary cluster definition.')
report_lines.append('')

report_lines.append('## Feature Cluster Interpretation')
for c in sorted(np.unique(feature_labels)):
    csize = int(np.sum(feature_labels == c))
    top_feats = feature_top[(feature_top['cluster'] == c) & (feature_top['rank_abs_delta'] <= 3)]
    top_txt = '; '.join([f"{r.feature} ({r.delta_z_vs_corpus:+.2f})" for r in top_feats.itertuples(index=False)])
    rep = rep_feature[rep_feature['cluster'] == c].iloc[0]
    report_lines.append(f"- Cluster {int(c)} (n={csize}): top signatures `{top_txt}`; representative `{int(rep['book_id'])} | {rep['title']}`.")
report_lines.append('')

report_lines.append('## Genre Composition by Cluster')
for c in genre_count_df.index:
    row_counts = genre_count_df.loc[c]
    top_genre = str(row_counts.idxmax())
    top_count = int(row_counts.max())
    top_prop = float(genre_prop_df.loc[c, top_genre])
    report_lines.append(f"- Cluster {int(c)} is most concentrated in `{top_genre}` ({top_count} books, {top_prop:.2%} of cluster).")
report_lines.append('')

report_lines.append('## Feature vs DTW Agreement')
report_lines.append(f'- ARI: `{ari:.3f}`')
report_lines.append(f'- NMI: `{nmi:.3f}`')
report_lines.append('- Interpretation: DTW captures broad trajectory shape, but does not strongly reproduce the feature-space grouping in this run.')
report_lines.append('')

report_lines.append('## Figure Gallery (Embedded)')
embedded_figures = [
    ('Feature PCA scatter (legend-checked)', 'feature_pca_scatter_feature_clusters.png'),
    ('KMeans elbow diagnostics', 'feature_elbow_kmeans_inertia.png'),
    ('Feature k-sweep quality metrics', 'feature_k_sweep_quality_metrics.png'),
    ('DTW k-sweep silhouette', 'dtw_k_sweep_silhouette.png'),
    ('Genre counts by feature cluster', 'genre_by_feature_cluster_counts.png'),
    ('Genre proportions by feature cluster', 'genre_by_feature_cluster_proportions.png'),
    ('Top feature signatures heatmap', 'cluster_feature_signature_heatmap_top12.png'),
    ('Feature vs DTW contingency heatmap', 'cluster_method_contingency_heatmap.png'),
    ('Feature cluster member trajectories (MA5)', 'feature_cluster_member_trajectories_ma5.png'),
    ('Feature/DTW centroid trajectories (raw)', 'cluster_centroid_trajectories_raw.png'),
    ('Feature/DTW centroid trajectories (MA5)', 'cluster_centroid_trajectories_ma5.png'),
    ('DTW distance heatmap', 'dtw_distance_heatmap.png'),
    ('DTW dendrogram', 'dtw_dendrogram.png'),
    ('Cluster size comparison', 'cluster_size_comparison.png'),
]
for title, fig_name in embedded_figures:
    report_lines.append(f'### {title}')
    report_lines.append(f'![{title}](figures/{fig_name})')
    report_lines.append('')

report_lines.append('## Practical Use / Caveats / Next Steps')
report_lines.append('- Use now: interpretable archetype grouping from feature clusters and MA5 pacing profiles.')
report_lines.append('- Use with caution: DTW-driven hard grouping, because selected DTW split is coarse in this run.')
report_lines.append('- Caveat: small sample size (20 books) means cluster boundaries should be treated as exploratory.')
report_lines.append('1. Add bootstrap re-sampling of books to quantify cluster stability confidence intervals.')
report_lines.append('2. Add constrained model selection that penalizes singleton clusters for DTW branch.')
report_lines.append('3. Evaluate agreement with external weak labels (genre/twist ranks) for external validity.')

cluster_report_path = OUT_ROOT / 'cluster_report.md'
cluster_report_path.write_text(chr(10).join(report_lines) + chr(10), encoding='utf-8')

# Write interim integrity so self-file existence can pass
pd.DataFrame(integrity_rows).to_csv(TABLE_DIR / 'integrity_checks.csv', index=False)

# Report image-link existence check
missing_report_figs = [name for _, name in embedded_figures if not (FIG_DIR / name).exists()]
integrity_rows.append({
    'check': 'cluster_report_embedded_figures_exist',
    'expected': len(embedded_figures),
    'actual': len(embedded_figures) - len(missing_report_figs),
    'pass': len(missing_report_figs) == 0,
})

# Output completeness
expected = expected_outputs()
missing = [str(p) for p in expected if not p.exists()]

integrity_rows.extend([
    {'check': 'output_files_complete', 'expected': len(expected), 'actual': len(expected) - len(missing), 'pass': len(missing) == 0},
    {'check': 'feature_assign_unique_books', 'expected': 20, 'actual': int(feature_assign_df['book_id'].nunique()), 'pass': int(feature_assign_df['book_id'].nunique()) == 20},
    {'check': 'dtw_assign_unique_books', 'expected': 20, 'actual': int(dtw_assign_df['book_id'].nunique()), 'pass': int(dtw_assign_df['book_id'].nunique()) == 20},
    {'check': 'ma_window_is_5', 'expected': 5, 'actual': MA_WINDOW, 'pass': MA_WINDOW == 5},
])

integrity_df = pd.DataFrame(integrity_rows)
integrity_df.to_csv(TABLE_DIR / 'integrity_checks.csv', index=False)

print('Saved insights:', OUT_ROOT / 'insights.md')
print('Saved cluster report:', cluster_report_path)
print('Saved integrity:', TABLE_DIR / 'integrity_checks.csv')
print('Integrity all-pass:', bool(integrity_df['pass'].all()))
if missing_report_figs:
    print('Missing report figures:', missing_report_figs)
if missing:
    print('Missing files:')
    for m in missing:
        print('-', m)
